## Vehicle Detection and Tracking Project

The goals / steps of this project are the following:

* Use color transforms, gradients, etc., to create a thresholded binary image.
* Detect vehicles on road.
* Output visual display of the detected cars.

---
## Camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import zipfile


from skimage.feature import hog
from skimage import data, exposure
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix

ImportError: dlopen(/Users/silviohohne/anaconda/envs/vehicle-detection/lib/python3.6/site-packages/PIL/_imaging.cpython-36m-darwin.so, 2): Symbol not found: _clock_gettime
  Referenced from: /Users/silviohohne/anaconda/envs/vehicle-detection/lib/python3.6/site-packages/PIL/.dylibs/liblzma.5.dylib (which was built for Mac OS X 10.12)
  Expected in: /usr/lib/libSystem.B.dylib
 in /Users/silviohohne/anaconda/envs/vehicle-detection/lib/python3.6/site-packages/PIL/.dylibs/liblzma.5.dylib

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    draw_img = np.copy(img)
    for (x1,y1),(x2,y2) in bboxes:
        cv2.rectangle(draw_img, (x1, y1), (x2, y2), color, thick)
    return draw_img # Change this line to return image copy with boxes



In [ ]:
def color_hist(image, nbins=32, bins_range=(0, 256)):
    rhist = np.histogram(image[:,:,0], bins=32, range=(0, 1.0))
    ghist = np.histogram(image[:,:,1], bins=32, range=(0, 1.0))
    bhist = np.histogram(image[:,:,2], bins=32, range=(0, 1.0))
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return rhist, ghist, bhist, bin_centers, hist_features

In [ ]:
if not os.path.isfile('data' + os.sep + 'vehicles' + os.sep + 'GTI_MiddleClose' + os.sep + 'image0003.png'):
    zip_ref = zipfile.ZipFile('data' + os.sep + 'vehicles.zip', 'r')
    zip_ref.extractall()
    zip_ref.close()
if not os.path.isfile('data' + os.sep + 'non-vehicles' + os.sep + 'GTI' + os.sep + 'image1.png'):
    zip_ref = zipfile.ZipFile('data' + os.sep + 'non-vehicles.zip', 'r')
    zip_ref.extractall()
    zip_ref.close()

In [ ]:
image = mpimg.imread('data/vehicles/GTI_MiddleClose/image0003.png')
_, _, _, _, hist_features = color_hist(image, nbins=32, bins_range=(0, 256))
#print ('image', image)
print ('hist_features', hist_features.shape, hist_features)

In [ ]:
def bin_spatial(img, color_space='RGB', size=(32, 32)):
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)             
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image, size).ravel() 
    # Return the feature vector
    return features


In [ ]:
# Define a function to return HOG features and visualization
# Features will always be the first element of the return
# Image data will be returned as the second element if visualize= True
# Otherwise there is no second return element
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=True, 
                     feature_vec=True):
                         
    """
    Function accepts params and returns HOG features (optionally flattened) and an optional matrix for 
    visualization. Features will always be the first return (flattened if feature_vector= True).
    A visualization matrix will be the second return if visualize = True.
    """
    
    return_list = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  block_norm= 'L2-Hys', transform_sqrt=False, 
                                  visualise= vis, feature_vector= feature_vec)
    
    # name returns explicitly
    hog_features = return_list[0]
    if vis:
        hog_image = return_list[1]
        return hog_features, hog_image
    else:
        return hog_features

In [ ]:
image = mpimg.imread('data/vehicles/GTI_MiddleClose/image0003.png')
hog_features = get_hog_features(image, orient=9, pix_per_cell=8, cell_per_block=2, vis=False, feature_vec=True)
print ('hog_features', hog_features)   

In [ ]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, orient=9, pix_per_cell=8, cell_per_block=2):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        _, _, _, _, hist_features = color_hist(image)
        hog_features = []
        hog_features.extend(hist_features.tolist())

        for colorspace in ['RGB','HSV','LUV','HLS','YUV', 'YCrCb']:
            feature_image = np.copy(image)
            if colorspace == 'HSV':
                feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2HSV)
            elif colorspace == 'LUV':
                feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2LUV)
            elif colorspace == 'HLS':
                feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2HLS)
            elif colorspace == 'YUV':
                feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2YUV)
            elif colorspace == 'YCrCb':
                feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2YCrCb)
            
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))

        hog_features = np.ravel(hog_features)        

        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

In [ ]:
features = extract_features(['./data/vehicles/GTI_MiddleClose/image0003.png'])
print ('features', features)

In [ ]:
def pipeline(img):
    return img

In [ ]:
def process_image(img):
    img_und = cv2.undistort(img, mtx, dist, None, mtx)
    bboxes = [((100, 100), (200, 200)), ((300, 300), (400, 400))]
    img_box = draw_boxes(img_und, bboxes)
    return img_box

In [ ]:
#img = mpimg.imread('test_images/test1.jpg')

#rh, gh, bh, bincen, feature_vec = color_hist(img, nbins=32, bins_range=(0, 256))
#fig = plt.figure(figsize=(12,3))
#plt.subplot(131)
#plt.bar(bincen, rh[0])
#plt.xlim(0, 256)
#plt.title('R Histogram')
#plt.subplot(132)
#plt.bar(bincen, gh[0])
#plt.xlim(0, 256)
#plt.title('G Histogram')
#plt.subplot(133)
#plt.bar(bincen, bh[0])
#plt.xlim(0, 256)
#plt.title('B Histogram')
#fig.tight_layout()


In [ ]:
#image = mpimg.imread('test_images/test1.jpg')
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Call our function with vis=True to see an image output
#features, hog_image = get_hog_features(gray, orient= 9, pix_per_cell= 8, cell_per_block= 2, vis=True, feature_vec=False)


# Plot the examples
#fig = plt.figure()
#plt.subplot(121)
#plt.imshow(image, cmap='gray')
#plt.title('Example Car Image')
#plt.subplot(122)
#plt.imshow(hog_image, cmap='gray')
#plt.title('HOG Visualization')


In [ ]:
def get_dataset(directory):
    car_filelist = []
    notcar_filelist = []
    
    for path, subdirs, files in os.walk(directory):
        for name in files:
            if name.endswith('.png'):
#                print (path, subdirs, name)
                if (os.sep + 'vehicles' + os.sep ) in path:
                    car_filelist.append(path + os.sep + name)
                if (os.sep + 'non-vehicles' + os.sep ) in path:
                    notcar_filelist.append(path + os.sep + name)

    return car_filelist, notcar_filelist


In [ ]:
car_filelist, notcar_filelist = get_dataset('./data')

In [ ]:
#print ('car', len(car_filelist), car_filelist)

In [ ]:
#print ('car', len(notcar_filelist), notcar_filelist)

In [ ]:
#print ('car')
#for i  in range(len(car_filelist)):
#    car_file =  car_filelist[i]
#    image = mpimg.imread(car_file)
#    print (car_file, image.shape)

In [ ]:
#for i  in range(len(car_filelist)):
#    car_file =  car_filelist[i]
#    image = mpimg.imread(car_file)
#    if i % 1090 == 34:
#        fig = plt.figure()
#        plt.imshow(image)

In [ ]:
#print ('notcar')
#for i in range(len(notcar_filelist)):
#    notcar_file =  notcar_filelist[i]
#    image = mpimg.imread(notcar_file)
#    print (notcar_file, image.shape)

In [ ]:
#for i  in range(len(notcar_filelist)):
#    notcar_file =  notcar_filelist[i]
#    image = mpimg.imread(notcar_file)
#    if i % 1000 == 34:
#        fig = plt.figure()
#        plt.imshow(image)

In [ ]:
X, y, mean, scale = None, None, None, None

X_train, X_test, y_train, y_test = None,  None,  None,  None

if os.path.isfile('dataset.npz'):
    print('dataset.npz exists')
    X_scaler = preprocessing.StandardScaler()
    dataset = np.load('dataset.npz')
    X, y, X_scaler.mean_ , X_scaler.scale_  = dataset['X'],  dataset['y'], dataset['mean'], dataset['scale']
else:
    print('dataset.npz new')
    car_features    = extract_features(car_filelist)
    notcar_features = extract_features(notcar_filelist)
    X = np.vstack((car_features, notcar_features)).astype(np.float64)
    y_car = np.ones((len(car_filelist)))
    y_notcar = np.zeros((len(notcar_filelist)))
    y = np.hstack((y_car, y_notcar)).astype(np.int)
    X_scaler = preprocessing.StandardScaler().fit(X)
    X = X_scaler.transform(X)
    np.savez_compressed('dataset.npz', X = X, y = y, mean = X_scaler.mean_, scale = X_scaler.scale_)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=452553)
print ('X_train.shape', X_train.shape)
print ('y_train.shape', y_train.shape)
print ('X_test.shape', X_test.shape)
print ('y_test.shape', y_test.shape)


In [ ]:
C_range = [0.235]
gamma_range = [0.03]
#C_range = [0.230, 0.235, 0.240, 0.245]
#gamma_range = [0.020, 0.025, 0.030, 0.035]
for C in C_range:
    for gamma in gamma_range:
        svc = svm.SVC(kernel='rbf', C=C, gamma=gamma)
#        svc = LinearSVC()
        svc.fit(X_train, y_train)
        print ('C', C, 'gamma', gamma)
        print ('score train', svc.score(X_train, y_train))
        print ('score test', svc.score(X_test, y_test))
        joblib.dump(svc, 'model.pkl') 
'''
C = 0.99 
gamma = 25.5
svc = svm.SVC(kernel='rbf', C=C, gamma=gamma)
svc.fit(X, y)

svc.score(X_train, y_train)
confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
'''
print ('training end')

In [ ]:
make_video = False
if make_video:
    white_output = 'output'  + os.sep + 'project_video.mp4'
    clip = VideoFileClip("project_video.mp4")
    white_clip = clip.fl_image(process_image) 
    %time white_clip.write_videofile(white_output, audio=False)